In [12]:
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
import pandas as pd
import json

def load_metric_from_event(event_name): 
    event_path = os.path.join("runs", event_name)
    event_name = os.listdir(event_path)[0]
    event_path = os.path.join(event_path, event_name)

    # only read scalars
    ea = event_accumulator.EventAccumulator(event_path, size_guidance={event_accumulator.SCALARS: 0})

    # load scalars
    ea.Reload()

    return get_best_epoch_metric(ea)

def get_best_epoch_metrics(ea):
    test_accs = ea.Scalars(f"test/acc")
    test_acc_vals = [metric.value for acc in test_accs]
    test_best_acc_idx = np.argmax(test_acc_vals)
    test_best_acc = np.max(test_acc_vals)
    test_best_epoch = test_accs[test_best_acc_idx].step

    train_accs = ea.Scalars(f"train/acc")
    train_best_acc = train_accs[test_best_acc_idx].value
    try:
        train_img_clas_accs = ea.Scalars(f"train/img_clas_acc")
        train_best_img_clas_acc = train_img_clas_accs[test_best_acc_idx].value

        
        test_img_clas_accs = ea.Scalars(f"train/img_clas_acc")
        test_best_img_clas_acc = test_img_clas_accs[test_best_acc_idx].value
    except KeyError:
        train_best_img_clas_acc = np.nan
        test_best_img_clas_acc = np.nan
    
    return [test_best_epoch, train_best_metric, test_best_metric, train_best_img_clas_acc, test_best_img_clas_acc]

## Define important variables

In [7]:
# events in tensorboard
events = ['24_03_2021_10_33_16_task_standard_seed_7', '24_03_2021_10_37_04_task_standard_seed_122', '24_03_2021_10_39_50_task_standard_seed_809', '24_03_2021_10_43_50_task_standard_seed_7', '24_03_2021_11_00_48_task_standard_seed_122', '24_03_2021_11_18_26_task_standard_seed_809', '24_03_2021_11_35_57_task_standard_seed_7', '24_03_2021_11_54_58_task_standard_seed_122', '24_03_2021_12_13_03_task_standard_seed_809', '24_03_2021_12_30_12_task_img_clas_seed_7', '24_03_2021_12_37_57_task_img_clas_seed_122', '24_03_2021_12_44_03_task_img_clas_seed_809', '24_03_2021_13_05_47_task_img_clas_seed_7', '24_03_2021_13_27_19_task_img_clas_seed_122', '24_03_2021_13_48_31_task_img_clas_seed_809', '24_03_2021_13_59_50_task_img_clas_seed_7', '24_03_2021_14_21_20_task_img_clas_seed_122', '24_03_2021_14_43_14_task_img_clas_seed_809', '24_03_2021_15_05_02_task_target_clas_seed_7', '24_03_2021_15_09_49_task_target_clas_seed_122', '24_03_2021_15_14_35_task_target_clas_seed_809', '24_03_2021_15_19_17_task_target_clas_seed_7', '24_03_2021_15_41_05_task_target_clas_seed_122', '24_03_2021_16_02_59_task_target_clas_seed_809', '24_03_2021_16_24_57_task_target_clas_seed_7', '24_03_2021_16_47_18_task_target_clas_seed_122', '24_03_2021_17_09_55_task_target_clas_seed_809']

# choose which columsn to include
include_cols = ["task", "ic_loss_weight", "num_imgs", "same_class_prob", "seed"]

In [10]:
results = []
args_dir = os.path.join("args")
for event_name in events:
    # get parameters
    with open(f"{args_dir}\\args_{event_name}.json") as json_file:
        params = json.load(json_file)
    # wrap each value into a list
    params = {key:[value] for key, value in params.items()}

    # create dataframe and add parameters
    results_df = pd.DataFrame(params)[include_cols]

    # add metrics
    metric = "acc"
    metric_cols = ["best epoch", f"best train {metric}", f"best test {metric}"]
    results_df.loc[:, metric_cols] = load_metric_from_event(event_name, metric)
    # append to final table
    results.append(results_df)

    metric = "img_clas_acc"
    metric_cols = [f"best train {metric}", f"best test {metric}"] # skip best epoch
    try:
        results_df.loc[:, metric_cols] = load_metric_from_event(event_name, metric)[1:]
    except KeyError:
        results_df.loc[:, metric_cols] = np.nan
    # append to final table
    results.append(results_df)

results = pd.concat(results)

In [11]:
results.drop(["ic_loss_weight", "num_imgs"], axis=1)

,task,same_class_prob,seed,best epoch,best train acc,best test acc,best train img_class_acc,best test img_class_acc
0,standard,0.0,7,NaN,0.96814,0.9756,NaN,NaN
0,standard,0.0,7,NaN,0.96814,0.9756,NaN,NaN
0,standard,0.0,122,NaN,0.95060,0.9709,NaN,NaN
0,standard,0.0,122,NaN,0.95060,0.9709,NaN,NaN
0,standard,0.0,809,NaN,0.96734,0.9748,NaN,NaN
0,standard,0.0,809,NaN,0.96734,0.9748,NaN,NaN
0,standard,0.5,7,NaN,0.94554,0.9609,NaN,NaN
0,standard,0.5,7,NaN,0.94554,0.9609,NaN,NaN
0,standard,0.5,122,NaN,0.91778,0.9361,NaN,NaN
0,standard,0.5,122,NaN,0.91778,0.9361,NaN,NaN
